<table align='right'><tr>
<td style="padding:10px"><img src="resources/img/logos/EC_POS.png" style="max-height:50px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/ESA_logo_2020_Deep.png" style="max-height:40px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/Copernicus_blue.png" style="max-height:60px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/AIRBUS_Blue.png" style="max-height:30px;width:auto;"/></td>
<td style="padding:10px"><img src="resources/img/logos/CS-GROUP.png" style="max-height:50px;width:auto;"/></td>
</tr></table>

<a href="./ESA_checkpoint_v0.1_02_manual_calls.ipynb" target="_blank"><< Part 2: manual calls to the RS-Server services</a>
<br>
<a href="./ESA_checkpoint_v0.1_04_prefect.ipynb" target="_blank">Part 4: Prefect workflows >></a>

<font color="#138D75">**Copernicus Reference System Python**</font> <br>
**Copyright:** Copyright 2024 ESA <br>
**License:** Apache License, Version 2.0 <br>
**Authors:** Airbus, CS Group

<div class="alert alert-block alert-success">
<h3>Copernicus Reference System Python tutorial for the ESA checkpoint 0.1</h3></div>

<div class="alert alert-block alert-warning">

<h4>Part 3: use the STAC catalog</h4>

Prerequisites:
* <a href="./ESA_checkpoint_v0.1_01_initialisation.ipynb" target="_blank">Part 1: initialisation</a>
* <a href="./ESA_checkpoint_v0.1_02_manual_calls.ipynb" target="_blank">Part 2: manual calls to the RS-Server services</a>

</div>
<hr>

# Introduction

## Links

* GitHub: https://github.com/RS-PYTHON
* Documentation: https://home.rs-python.eu/rs-documentation/

## Data used

In this notebook, we use simulated Auxip and Cadip data.

## Learning outcomes

At the end of this notebook you will know how to:
* Use the RS-Client Python library.
* Use it to call the STAC catalog services.

<div class="alert alert-info" role="alert">

## Contents

</div>
    
1. [Check your installation](#Check-your-installation) 
1. [RsClient initialisation](#RsClient-initialisation)
1. [Use the STAC catalog](#Use-the-STAC-catalog)
1. [Exercises](#Exercises)

<hr>

<div class="alert alert-info" role="alert">

## Check your installation

In this section, we will check that your Jupyter Notebook environment is correctly set.

[Back to top](#Contents)

</div>

### `rs-client-libraries` installation

The `rs-client-libraries` Python library is the preferred way to access the RS-Server services from your environment. It is automatically installed in this notebook.

**Note**: don't worry about these OpenTelemetry messages for now, they will be fixed in a later version:
```
Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented
Transient error StatusCode.UNAVAILABLE encountered while exporting metrics to ..., retrying in ...s
Failed to export metrics to ..., error code: StatusCode.UNIMPLEMENTED
```

In [ ]:
import rs_client
import rs_common
import rs_workflows

# Set logger level to info
import logging
rs_common.logging.Logging.level = logging.INFO

### Environment

In [ ]:
import os

# In local mode, all your services are running locally.
# In hybrid or cluster mode, we use the services deployed on the RS-Server website.
# This configuration is set in an environment variable.
local_mode = (os.getenv("RSPY_LOCAL_MODE") == "1")

# In local mode, the service URLs are hardcoded in the docker-compose file
if local_mode:
    rs_server_href = None # not used
    RSPY_HOST_AUXIP = "http://localhost:8001/docs"
    RSPY_HOST_CADIP = "http://localhost:8002/docs"
    RSPY_HOST_CATALOG = "http://localhost:8003/api.html"

# In hybrid or cluster mode, they are set in an environment variables
else:
    rs_server_href = os.environ["RSPY_WEBSITE"]

### API key

In [ ]:
apikey = os.getenv("RSPY_APIKEY")
if (not local_mode) and (not apikey):
    import getpass
    apikey = getpass.getpass(f"Enter your API key:")
    os.environ["RSPY_APIKEY"] = apikey

<div class="alert alert-info" role="alert">

## RsClient initialisation

Initialise Python RsClient class instances to access the RS-Server services.

[Back to top](#Contents)

</div>

In [ ]:
import json
from rs_client.rs_client import RsClient
from rs_common.config import ECadipStation

# Init a generic RS-Client instance. Pass the:
#   - RS-Server website URL
#   - API key. If not set, we try to read it from the RSPY_APIKEY environment variable.
#   - ID of the owner of the STAC catalog collections.
#     By default, this is the user login from the keycloak account, associated to the API key.
#     Or, in local mode, this is the local system username.
#     Else, your API Key must give you the rights to read/write on this catalog owner (see next cell).
#   - Logger (optional, a default one can be used)
generic_client = RsClient(rs_server_href, rs_server_api_key=None, owner_id=None, logger=None)
print(f"STAC catalog owner: {generic_client.owner_id!r}")

# From this generic instance, get an Auxip client instance
auxip_client = generic_client.get_auxip_client()

# Or get a Cadip client instance. Pass the cadip station.
cadip_station = ECadipStation.CADIP # you can also have: INS, MPS, MTI, NSG, SGS
cadip_client = generic_client.get_cadip_client(cadip_station)

# Or get a Stac client to access the catalog
stac_client = generic_client.get_stac_client()

print("\nValidate that our catalog is valid to the STAC format...")
stac_client.validate_all()

print("\nDisplay the Stac catalog as a treeview in notebook:")
display(stac_client)

print("\nOr just display all its contents at once:")
print(json.dumps(stac_client.to_dict(), indent=2))

In [ ]:
# In hybrid or cluster mode, show information from the keycloak account, associated to the api key
if not local_mode:

    # = keycloak account user login
    print(f"API key user login: {generic_client.apikey_user_login!r}")

    # Print the IAM (Identity and Access Management) roles
    # For this tutorial, you must have: 
    #   - read/download access for Adgs (=Auxip) = "rs_adgs_<read|download>"
    #   - read/download access to the Cadip station you passed on the above cell = "rs_cadip_<station>_<read|download>"
    #   - (optional) read/write/download access to STAC catalog collections from other owners = "rs_catalog_<owner_id>:<collection|*>_<read|write|download>"
    #     (you always have all access to your own collections with owner_id=apikey_user_login as printed above)
    iam_roles = "\n".join (sorted (generic_client.apikey_iam_roles))
    print(f"\nAPI key IAM roles: \n{iam_roles}")

<div class="alert alert-info" role="alert">

## Use the STAC catalog

The SpatioTemporal Asset Catalog (STAC) family of specifications aim to standardize the way geospatial asset metadata is structured and queried. 

A 'spatiotemporal asset' is any file that represents information about the Earth captured in a certain space and time. 

For more information, see: https://github.com/radiantearth/stac-api-spec/tree/main

In this section, we will see how to use most of the RS-Server STAC catalog functionalities.

[Back to top](#Contents)

</div>

In [ ]:
# Do some initialisation
from datetime import datetime
import json

# Define a search interval
start_date = datetime(2010, 1, 1, 12, 0, 0)
stop_date = datetime(2024, 1, 1, 12, 0, 0)

#### Add a new collection to the catalog

In [ ]:
from pystac import Collection, Extent, SpatialExtent, TemporalExtent

COLLECTION = "my_tutorial_collection"

# Clean the existing collection, if any
stac_client.remove_collection(COLLECTION)

# In this tutorial, after each operation, we will validate that 
# our catalog is valid to the STAC format, but this is optional.
stac_client.validate_all()

# Add new collection 
response = stac_client.add_collection(
    Collection(
        id=COLLECTION,
        description=None, # rs-client will provide a default description for us
        extent=Extent(
            spatial=SpatialExtent(bboxes=[-180.0, -90.0, 180.0, 90.0]),
            temporal=TemporalExtent([start_date, stop_date])
        )
    ))
response.raise_for_status()
stac_client.validate_all()

#### Read collections from the catalog

In [ ]:
# See all my personal catalog collections
for collection in stac_client.get_collections():
    print(f"I have collection: {collection} at {collection.self_href}")

# Get a specific collection information
my_collection = stac_client.get_collection(collection_id=COLLECTION)
print(f"\nCollection information from {my_collection.self_href}\n{json.dumps(collection.to_dict(), indent=2)}")

#### Add new items to the collection

When RS-Server stages a file, it means to:
1. Copy (=download) it from the reception station into the temporary S3 bucket.
1. Publish its metadata into the STAC catalog and move it from the temporary into the final S3 bucket.

The first step has been done in the previous tutorial. Here we will perform the second step.

<mark>WARNING: after this, the staged files are moved from the temporary into the final bucket, so this cell can be run only once, or you'll have to stage the files again from the previous tutorial.</mark>

![Staging step #2](resources/img/v0.1/staging_step2.drawio.png)

In [ ]:
from pystac.asset import Asset
from pystac.item import Item

# Simulated values
WIDTH=2500
HEIGHT=2500

# We will add one Auxip and one Cadip file that were staged from the previous notebook
%store -r temp_s3_files
for temp_s3_file in temp_s3_files:

    # Let's use STAC item ID = filename
    print(f"Add catalog item from: {temp_s3_file!r}")
    item_id = os.path.basename(temp_s3_file)

    # The file path from the temp s3 bucket is given in the assets
    assets = {"file": Asset(href=temp_s3_file)}

    # Other hardcoded parameters for this demo
    geometry = {
        "type": "Polygon",
        "coordinates": [[[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]],
    }
    bbox = [-180.0, -90.0, 180.0, 90.0]
    now = datetime.now()
    properties = {
        "gsd": 0.12345,
        "width": WIDTH,
        "height": HEIGHT,
        "datetime": datetime.now(),
        "proj:epsg": 3857,
        "orientation": "nadir",
    }

    # Add item to the STAC catalog collection, check status is OK
    # NOTE: in future versions, this pystac Item object will be returned automatically by rs-client-libraries.
    item = Item(
        id=item_id,
        geometry=geometry,
        bbox=bbox,
        datetime=now,
        properties=properties,
        assets=assets)
    response = stac_client.add_item(COLLECTION, item)
    response.raise_for_status()
    stac_client.validate_all()

#### Read items from the collection

In [ ]:
# Get the items from the catalog to check that they were inserted
for temp_s3_file in temp_s3_files:
    item_id = os.path.basename(temp_s3_file)
    inserted_item = my_collection.get_item(item_id)
    assert inserted_item, "Item was not inserted"
    print (f"Saved item in the catalog:\n{json.dumps (inserted_item.to_dict(), indent=2)}")

#### Search items from the catalog

In [ ]:
# For searching, we need to prefix our collection name by <owner_id>_
owner_collection = f"{stac_client.owner_id}_{COLLECTION}"

# Search by the last inserted item id
search = stac_client.search(ids=[item_id], collections=[owner_collection])
results = list(search.items_as_dicts())
assert results, f"There should be at least one item with id={item_id}"
print(f"Found {len(results)} results for id={item_id}")

# Search by the 'width' and 'height' property using a CQL2 filter, 
# see: https://pystac-client.readthedocs.io/en/stable/tutorials/cql2-filter.html
filter_on_dimensions = {
    "op": "and",
    "args": [
        {"op": "=", "args" : [{"property": "collection"}, owner_collection]},
        {"op": "=", "args" : [{"property": "width"}, WIDTH]},
        {"op": "=", "args" : [{"property": "height"}, HEIGHT]},
    ]
}
search = stac_client.search(filter=filter_on_dimensions)
results = list(search.items_as_dicts())
assert results, f"There should be at least one item for width={WIDTH} height={HEIGHT}"
print(f"\nFound {len(results)} results for width={WIDTH} height={HEIGHT}")
for result in results:
    print(f"({result['collection']}) {result['id']}")

#### Remove an item from the collection

In [ ]:
# Get all items before removing
items_before = list(stac_client.get_collection(COLLECTION).get_items())
print (f"{len(items_before)} items before removing")

# If there is at least one item
if items_before:

    # Remove the first item
    item_id = items_before[0].id
    stac_client.remove_item (COLLECTION, item_id)
    
    # We should have one less item in the collection
    items_after = list(stac_client.get_collection(COLLECTION).get_items())
    assert len(items_after) == (len(items_before) - 1), \
        f"There should be {len(items_before) - 2} items in the collection, but we have {len(items_after)}"

stac_client.validate_all()

#### Remove a collection from the catalog

In [ ]:
import pystac_client

# Remove the collection
stac_client.remove_collection (COLLECTION)

# It should not exist anymore: trying to get the collection should raise an Exception
try:
    stac_client.get_collection(COLLECTION)
    assert False, f"The collection {COLLECTION!r} should have been removed"

# So it is normal that we have this exception
except pystac_client.exceptions.APIError:
    print (f"The collection {COLLECTION!r} has been removed")

stac_client.validate_all()

<div class="alert alert-danger" role="alert">

## Exercises

</div>

Run again the previous cells but this time: 
1. Check if your **API key** allows you to access STAC catalog collections from **other owners**. Then:
    1. Use a RsClient instance with one of these **other owner IDs**.
    1. Check that this **other owner ID** is saved as the **owner** field of your newly created collections.
1. Using the previous notebook, stage at least one different Auxip and Cadip file into the STAC catalog.

**NOTE**: you can also use the website OpenAPI Swagger UI to call RS-Server (see cell below).

[Back to top](#Contents)

<hr>

In [ ]:
if local_mode:
    print(f"""OpenAPI Swagger UI for:
  - Auxip: {RSPY_HOST_AUXIP}
  - Cadip: {RSPY_HOST_CADIP}
  - STAC catalog: {RSPY_HOST_CATALOG}""")
else:
    print(f"OpenAPI Swagger UI: {generic_client.rs_server_href}")

<a href="./ESA_checkpoint_v0.1_02_manual_calls.ipynb" target="_blank"><< Part 2: manual calls to the RS-Server services</a>
<br>
<a href="./ESA_checkpoint_v0.1_04_prefect.ipynb" target="_blank">Part 4: Prefect workflows >></a>

<hr>
<a href="https://github.com/RS-PYTHON" target="_blank">View on GitHub</a>